# Séance 2 : MLMC estimator (scalar version)

We want to implement in python the MLMC estimator of the expectation ([Myzek & de Lozzo](https://hal.science/hal-01894503/document), (2.12)) with $f$ a function that returns a scalar value, defined as here for $d \in \mathbb{N}$ : 
$$
\begin{align*}
    f : \mathbb{R}^d & \rightarrow \mathbb{R} \\
    X & \mapsto v^T \cdot X, \qquad v \in \mathbb{R}^d, 
\end{align*}
$$

## Variables definition
Let $X \sim \mathcal{N}_d(0,1)$, $v_1 \in \mathbb{R}^d$ as $f(X) = v_1^T \cdot X$ with $v_1$ the **high fidelity** coefficent vector.

We have : 
$$
\mathbb{E}\left[ X \right] = 0 \\
\mathbb{V}\left( X \right) = 1$$

and :
$$
\mathbb{E}\left[ f(X) \right] = 0 \\
\mathbb{V}\left( f(X) \right) = \mathbb{V}\left( \sum_{i=1}^d v_i X_i \right) = \sum_{i=1}^d v_i^2 \mathbb{V}\left( X_i \right) = \sum_{i=1}^d v_i^2 $$
because $X_i$ are i.i.d.

## Classic estimators
Let $(X_n)_{n\in\mathbb{N}} \in \mathbb{R}^d$ a sequence of random variables i.i.d following the normal distribution $\mathcal{N}_d(0,1)$.

We have :
$$
\hat{\mathbb{E}}\left[ f(X) \right] = \frac{1}{n} \sum_{i=1}^n f(X) \\
\hat{\mathbb{V}}\left( f(X) \right) = \mathbb{E}\left[ f(X)^2 \right] \approx \frac{1}{n} \sum_{i=1}^n f(X)^2
$$

because $\mathbb{E}\left[ f(X) \right]^2 = 0$.


### Code

First, let's code our functions, and our variables :

In [ ]:
import numpy as np 

In [ ]:
# Constants 
d = 10

#print(np.random.uniform(-1,1,d))

v_1= np.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773])

In [ ]:
# Define f

def f(X,v):
    return v.T@X

In [ ]:
# Testing f
X = np.random.standard_normal(10)
print(f(X,v_1))

Now let's estimate the expectation and the variance of $f(X)$ for several $X_i$.

In [ ]:
n=100
X = np.random.standard_normal((d,n))

In [ ]:
# print(X)

In [ ]:
def f_vec(X,v):
    """
    X : (d,n) matrix
    v : d vector 
    """
    return v.T@X

def f_vec_squared(X,v):
    """
    X : (d,n) matrix
    v : d vector 
    """
    return (v.T@X)**2

exp_var = np.sum(np.square(v_1))

In [ ]:
# Estimation of the expecation and the variance :
E_MC = np.mean(f_vec(X,v_1))
Var_MC = np.mean(f_vec_squared(X,v_1))
print("Expectation = ", E_MC)
print("Variance = ", Var_MC)
print("Expected variance (sum of v_i^2) = ", exp_var)

For n = 100, we are quite good with the standard MC method.

## MLMC estimation
We still have $(X_n)_{n\in\mathbb{N}} \in \mathbb{R}^d$ a sequence of random variables i.i.d following the normal distribution $\mathcal{N}_d(0,1)$. 

We still have $v_1 \in \mathbb{R}^d$ for our function $f_1$ (previous $f$). We now have a low fidelity function $f_0$ based on the high fidelity function $f_1$ with $v_0 = v_1 + \varepsilon$, with $\varepsilon \sim \mathcal{N}_d(0_d,\sigma^2 I_d)$ and $\sigma^2 = 0.01$ or $0.1$. 

We now have :

$$
\hat{\mathbb{E}}\left[ f_1(X) \right] = \frac{1}{n_0} \sum_{k=1}^{n_0} f_0(X^{(k,0)}) + \frac{1}{n_1} \sum_{k=1}^{n_1} \left[f_1(X^{(k,1)})-f_0(X^{(k,1)})\right]
$$

We have aswell for the variance : 
$$
\begin{align}
\hat{\mathbb{V}}\left( f_1(X) \right) &= \mathbb{E}\left[ f_1(X)^2 \right] \\
& \approx \frac{1}{n_0} \sum_{k=1}^{n_0} f_0(X^{(k)})^2 + \frac{1}{n_1} \mathbb{V}\left( f_1(X) - f_0(X) \right) \leftarrow \text{pas bon !} \\
& \approx \frac{1}{n_0} \sum_{k=1}^{n_0} f_0(X^{(k,0)})^2 + \frac{1}{n_1} \sum_{k=1}^{n_1} \left[f_1(X^{(k,1)})^2 -f_0(X^{(k,1)})^2 \right] \\
\end{align}
$$

with : 
$$
\begin{align}
\mathbb{V}\left( f_\ell(X) - f_{\ell-1}(X) \right) & = \mathbb{E}\left[ \left( f_\ell(X) - f_{\ell-1}(X) \right)^2 \right] - \mathbb{E}\left[ f_\ell(X) - f_{\ell-1}(X) \right]^2 \\
& = \mathbb{E}\left[ \left( f_\ell(X) - f_{\ell-1}(X) \right)^2 \right] - \left(\mathbb{E}\left[ f_\ell(X)\right]-\mathbb{E}\left[ f_{\ell-1}(X) \right] \right)^2 \\
&= \mathbb{E}\left[ \left( f_\ell(X) - f_{\ell-1}(X) \right)^2 \right] \\
\end{align}  
$$

cause $ \mathbb{E}\left[ f_\ell(X)\right] = 0, \; \forall \ell \in \{0,...,L\} $ 


### Code

Let's code our variables :

In [ ]:
d = 10 # lenght of random vectors
print("sample size n =", n)

In [ ]:
eps = np.random.normal(0,0.01,d)
# print(eps)

v_0 = v_1 + eps

In [ ]:
n0 = 100
n1 = 10
X1 = np.random.standard_normal((d,n1))
X0 = np.random.standard_normal((d,n0))

In [ ]:
# Calcul de E_MLMC
Y1_1 = f_vec(X1,v_1)
Y1_0 = f_vec(X1,v_0)
Y0_0 = f_vec(X0,v_0)
E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)
print("E_MLMC =", E_MLMC)

In [ ]:
# Calcul de Var_MLMC
#  3.7900798793633337
Y0_0_squared = f_vec_squared(X0,v_0)
Y1_0_squared = f_vec_squared(X1,v_0)
Y1_1_squared = f_vec_squared(X1,v_1)
Var_MLMC = np.mean(Y0_0_squared) + np.mean(Y1_1_squared-Y1_0_squared)
print("Var_MLMC =", Var_MLMC)

In [ ]:
# Comparaison 
Err_Var_MC = np.abs(Var_MC-exp_var)/exp_var * 100
Err_Var_MLMC = np.abs(Var_MLMC-exp_var)/exp_var * 100

print("Error on the Var_MC =",Err_Var_MC, "%\nError on the Var_MLMC =", Err_Var_MLMC,"%")